In [1]:
import cv2
import numpy as np

In [2]:
# 사용할 모델 파일명
model_name = 'opencv_data/res10_300x300_ssd_iter_140000.caffemodel'
# 알고리즘 구조를 정의한 파일
protoextName = 'opencv_data/deploy.prototxt.txt'

# 사용할 이미지
# file_name = 'opencv_data/image/marathon_01.jpg'
# file_name = 'opencv_data/image/marathon_02.jpg'
file_name = 'opencv_data/image/marathon_03.jpg'
file_name

'opencv_data/image/marathon_03.jpg'

In [4]:
# 이미지를 읽어온다.
frame = cv2.imread(file_name)

In [4]:
# 사용할 모델을 불러온다.
model = cv2.dnn.readNetFromCaffe(protoextName, model_name)
model

<dnn_Net 0000022479C7DF90>

In [5]:
# 데이터를 변환한다.
# 이 모델은 300x300 짜리 이미지 14만장을 학습한 이미지이기 때문에
# 사이즈를 동일하게 맞춰줘야 한다.
a1 = cv2.resize(frame, (300, 300))

In [6]:
# 2진 데이터로 변환한다.
# 원본 이미지 데이터, 스케일링(크기조정), 결과 데이터 행렬 사이즈,
# 표준화를 위해 기준이 되는 색상값
blob = cv2.dnn.blobFromImage(a1, 1.0, (300, 300), (104.0, 177.0, 123.0))
blob

array([[[[151., 151., 151., ..., 151., 151., 151.],
         [151., 151., 151., ..., 151., 151., 151.],
         [148., 145., 147., ..., 150., 151., 151.],
         ...,
         [150., 150., 151., ..., 151., 148., 151.],
         [151., 148., 146., ..., 149., 151., 151.],
         [145., 149., 149., ..., 147., 148., 151.]],

        [[ 77.,  77.,  78., ...,  78.,  78.,  78.],
         [ 78.,  78.,  78., ...,  78.,  78.,  78.],
         [ 78.,  75.,  76., ...,  77.,  78.,  78.],
         ...,
         [ 77.,  77.,  78., ...,  78.,  75.,  78.],
         [ 78.,  75.,  73., ...,  76.,  78.,  78.],
         [ 72.,  76.,  76., ...,  75.,  73.,  78.]],

        [[132., 132., 132., ..., 132., 132., 132.],
         [131., 131., 131., ..., 132., 132., 132.],
         [130., 127., 129., ..., 131., 132., 132.],
         ...,
         [131., 131., 132., ..., 130., 131., 132.],
         [132., 129., 127., ..., 130., 132., 132.],
         [126., 130., 130., ..., 129., 126., 132.]]]], dtype=float32)

In [7]:
# 학습 모델에 데이터를 넣어준다.
model.setInput(blob)
# 얼굴 부분을 인식한다.
detections = model.forward()
detections.shape

(1, 1, 200, 7)

In [8]:
# 얼굴이라고 인식되는 부분의 개수만큼 반복한다.
# 결과의 3번째가 인식된 얼굴의 개수
for i in range(0, detections.shape[2]) :
    
    # 얼굴이라고 인지한 정확도를 가져온다.
    confidence = detections[0, 0, i, 2]
    # print(confidence)
    
    # 확률이 높은 것만 표시한다.
    if confidence > 0.2 :
        # 얼굴이라고 인지된 부분을 모두 표시한다.

        # 얼굴 부분 데이터를 가져온다.(이미지상의 비율)
        box1 = detections[0, 0, i, 3:7]
        # 원본 이미지에 맞게끔 좌표를 계산한다.
        w1 = frame.shape[1]
        h1 = frame.shape[0]
        # box1은 좌측상단 x,y 우측하단 x,y 좌표 비율이기에
        # 가로 세로길이를 곱해서 좌표를 환산한다.
        box2 = box1 * np.array([w1, h1, w1, h1])

        # 정수로 변환한다.
        box3 = box2.astype('int')

        # 네모를 그린다.
        cv2.rectangle(frame, (box3[0], box3[1]), (box3[2], box3[3]), (0, 255, 0), 2)
    
cv2.imshow('result', frame)
cv2.waitKey()
cv2.destroyAllWindows()